In [ ]:
# !pip install pandas numpy scikit-learn nltk 

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle 

In [2]:
import nltk

nltk.download("stopwords", quiet=True)
english_stopwords = set(nltk.corpus.stopwords.words('english'))

## Data Preprocessing

In [3]:
df = pd.read_csv("data/movie_dataset.csv")
df.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [4]:
df.isna().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [5]:
df["genres"].fillna("", inplace=True)
df["keywords"].fillna("", inplace=True)
df["tagline"].fillna("", inplace=True)

/var/folders/s5/sr_gfnts5zbglbz69w2_mdkh0000gn/T/ipykernel_88354/892714436.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["genres"].fillna("", inplace=True)
/var/folders/s5/sr_gfnts5zbglbz69w2_mdkh0000gn/T/ipykernel_88354/892714436.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as

In [6]:
df['overview'] = df['genres'] + ' ' + df['keywords'] + ' ' + df['overview'] + ' ' + df['tagline']
df['overview']

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      When ambitious New York attorney Sam is sent...
4802    Documentary obsession camcorder crush dream gi...
Name: overview, Length: 4803, dtype: object

In [7]:
df = df[["original_title", "overview"]]
df.isna().sum()

original_title    0
overview          3
dtype: int64

In [8]:
df.dropna(inplace=True)
df

,original_title,overview
0,Avatar,Action Adventure Fantasy Science Fiction cultu...
1,Pirates of the Caribbean: At World's End,Adventure Fantasy Action ocean drug abuse exot...
2,Spectre,Action Adventure Crime spy based on novel secr...
3,The Dark Knight Rises,Action Crime Drama Thriller dc comics crime fi...
4,John Carter,Action Adventure Science Fiction based on nove...
...,...,...
4798,El Mariachi,Action Crime Thriller united states\u2013mexic...
4799,Newlyweds,Comedy Romance A newlywed couple's honeymoon ...
4800,"Signed, Sealed, Delivered",Comedy Drama Romance TV Movie date love at fir...
4801,Shanghai Calling,When ambitious New York attorney Sam is sent...


## Feature Construction

In [9]:
tfidf_vectorizer = TfidfVectorizer(
    tokenizer = nltk.word_tokenize,
    stop_words = list(english_stopwords),
    ngram_range = (1,2),
    max_df = 1.0,
    min_df = 10
)

features = tfidf_vectorizer.fit_transform(df["overview"])

names = df["original_title"].values

names.shape, features.shape

/Users/frostday/miniconda3/envs/temp/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/frostday/miniconda3/envs/temp/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn(


((4800,), (4800, 4493))

In [10]:
pickle.dump(tfidf_vectorizer, open("saved/vectorizer.pkl", "wb"))

## Inference

In [11]:
tfidf_vectorizer = pickle.load(open("saved/vectorizer.pkl", "rb"))

In [12]:
def get_recommendations(text, top=5):
    X = tfidf_vectorizer.transform([text])
    similarities = cosine_similarity(X, features)[0]
    indices = np.argsort(similarities)[-top:][::-1]
    print("Recommended Movies:")
    recs = [f"{names[i]} ({similarities[i]:.2f})" for i in indices]
    print(' | '.join(recs))

In [13]:
get_recommendations("I love thrilling action movies set in space, with a comedic twist.")

Recommended Movies:
Moonraker (0.22) | Lost in Space (0.20) | Love and Death on Long Island (0.19) | Grindhouse (0.19) | Space Chimps (0.19)


In [14]:
get_recommendations("I like horror movies with ghosts and demons.")

Recommended Movies:
Grindhouse (0.24) | Thir13en Ghosts (0.22) | Demonic (0.18) | Goosebumps (0.18) | Insidious: Chapter 3 (0.18)


In [23]:
get_recommendations("I enjoy historical romance genre.", 10)

Recommended Movies:
Better Luck Tomorrow (0.27) | Cleopatra (0.20) | Melancholia (0.19) | State Fair (0.19) | GirlHouse (0.17) | Nomad (0.17) | Elsa & Fred (0.17) | Me and Orson Welles (0.16) | The Last Station (0.15) | Elizabeth (0.15)
